In [1]:
_w = f"""
{a}
{b}
"""

NameError: name 'a' is not defined

In [18]:
import string

template = string.Template("""
${a}
${b}
""")
partial_str = template.safe_substitute({'a': 1})  # '1 ${y}'
final_str = string.Template(partial_str).safe_substitute({'b': 2})  # '1 2'

print(partial_str)
print(final_str)


1
${b}


1
2



In [15]:
"""
1
2
"""

'\n1\n2\n'

In [1]:
s = '{foo} {bar}'
partial = s.format(foo='FOO')  # 'FOO {bar}'

KeyError: 'bar'

In [6]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow

@workflow()
def add(arg1: float, arg2: float) -> float:
    res = arg1+arg2
    return res

result = add(arg1=1, arg2=2)
workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl['__root__']

TaskException: can only concatenate tuple (not "dict") to tuple

In [7]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow


@task()
def sum(arg1: float, arg2: float) -> float:
    res = arg1+arg2
    return res

@workflow()
def add(arg1: float, arg2: float) -> float:
    x, y = arg1, arg2
    res = sum(arg1=x, arg2=y)
    return res

result = add(arg1=1, arg2=2)
workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
# cwl['__root__']
cwl

{'__root__': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {'add-1-arg2': {'label': 'arg2', 'type': 'int', 'default': 2},
   'add-1-arg1': {'label': 'arg1', 'type': 'int', 'default': 1}},
  'outputs': {'out': {'label': 'out',
    'type': 'float',
    'outputSource': 'add-1/out'}},
  'steps': {'add-1': {'run': 'add',
    'in': {'arg2': {'source': 'add-1-arg2'}, 'arg1': {'source': 'add-1-arg1'}},
    'out': ['out']}}},
 'add-1': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {'arg2': {'label': 'arg2', 'type': 'int', 'default': 2},
   'arg1': {'label': 'arg1', 'type': 'int', 'default': 1}},
  'outputs': {'out': {'label': 'out',
    'type': 'float',
    'outputSource': 'sum-1-1/out'}},
  'steps': {'sum-1-1': {'run': 'sum',
    'in': {'arg1': {'source': 'arg1'}, 'arg2': {'source': 'arg2'}},
    'out': ['out']}}}}

In [8]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow
from dewret.annotations import AtRender

@task()
def concat_strings(str1: str, str2: str) -> str:
    return str1 + str2

@workflow()
def create_greeting_in_all_caps(prefix: AtRender[str], name: AtRender[str]) -> str:
    prefix_cap = prefix.capitalize()
    name_cap = name.capitalize()
    return concat_strings(str1=prefix_cap, str2=name_cap)

result = create_greeting_in_all_caps(prefix="Hello to ", name="John")
workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl

{'__root__': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'string',
    'outputSource': 'create_greeting_in_all_caps-1/out'}},
  'steps': {'create_greeting_in_all_caps-1': {'run': 'create_greeting_in_all_caps',
    'in': {'prefix': {'default': 'Hello to '}, 'name': {'default': 'John'}},
    'out': ['out']}}},
 'create_greeting_in_all_caps-1': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'string',
    'outputSource': 'concat_strings-1-1/out'}},
  'steps': {'concat_strings-1-1': {'run': 'concat_strings',
    'in': {'str1': {'default': 'Hello to '}, 'str2': {'default': 'John'}},
    'out': ['out']}}}}

In [1]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow

@task()
def concat_strings(str1: str, str2: str) -> str:
    return str1 + str2

@task()
def cap(arg: str) -> str:
    return arg.capitalize()

@workflow()
def create_greeting_in_all_caps(prefix: str, name: str) -> str:
    prefix_cap = cap(arg=prefix)
    name_cap = cap(arg=name)
    return concat_strings(str1=prefix_cap, str2=name_cap)


result = create_greeting_in_all_caps(prefix="Hello to ", name="John")

workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl

ModuleNotFoundError: No module named 'dewret'

In [3]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow

@task()
def f(a: int, b: int,c: int) -> int:
    return a

@workflow()
def foo() -> int:
    return f(**{'a':1, 'b': 2, 'c': 3})

result = foo()

workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl

{'__root__': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'int',
    'outputSource': 'foo-1/out'}},
  'steps': {'foo-1': {'run': 'foo', 'in': {}, 'out': ['out']}}},
 'foo-1': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'int',
    'outputSource': 'f-1-1/out'}},
  'steps': {'f-1-1': {'run': 'f',
    'in': {'a': {'default': 1}, 'b': {'default': 2}, 'c': {'default': 3}},
    'out': ['out']}}}}

In [2]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow

@task()
def f(a: int, b: int,c: int) -> int:
    return a

@workflow()
def foo() -> int:
    return f(**{'a':1, 'b': 2, 'c': 3})

result = foo()

workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl

{'__root__': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'int',
    'outputSource': 'foo-1/out'}},
  'steps': {'foo-1': {'run': 'foo', 'in': {}, 'out': ['out']}}},
 'foo-1': {'cwlVersion': 1.2,
  'class': 'Workflow',
  'inputs': {},
  'outputs': {'out': {'label': 'out',
    'type': 'int',
    'outputSource': 'f-1-1/out'}},
  'steps': {'f-1-1': {'run': 'f',
    'in': {'a': {'default': 1}, 'b': {'default': 2}, 'c': {'default': 3}},
    'out': ['out']}}}}

In [ ]:
from dewret.renderers.cwl import render
from dewret.tasks import task, construct, workflow
from typing import TypedDict, NotRequired, Unpack, Any

@task()
def f(a: int, b: int,c: int) -> list[int]:
    return [a, b, c]

def g(**Unpack[])


@workflow()
def foo() -> None:
    f(**{'a':1, 'b': 2, 'c': 3})



result = foo()

workflow = construct(result, simplify_ids=True)
cwl = render(workflow)
cwl

TaskException: Attempted to build a workflow from a return-value/result/expression with no references.

In [26]:
file, module, pipeline_name = None, 1, 1
# "test_pipeline"

match file, module, pipeline_name:
    case None, None, None:
        print("fail - all None")
    case file, None, None:
        print("pass - only file")
    case (None, _, None) | (None, None, _):
        print("fail - need m and p")
    case (_, _, None) | (_, None, _):
        print("fail - need m or f")
    case None, module, pipeline_name:
        print(f"pass - file is {file}, module is {module} and pipeline_name is {pipeline_name}")

pass - file is None, module is 1 and pipeline_name is 1


In [53]:
file, module, pipeline_name = 1, None, 1

(bool(file) or (bool(module) == bool(pipeline_name)))


True

In [46]:
print (bool(1))

True
